In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import acnportal
import sklearn
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import seaborn as sns

from copy import deepcopy
import warnings
import pytz
import time
import numpy as np
import pandas as pd
import pickle
from datetime import datetime, timedelta
from enum import Enum
from collections import namedtuple, defaultdict
import gzip
import random
import os
import json

from acnportal import acnsim
from acnportal import algorithms
from acnportal.acnsim.events import EventQueue
# from acnportal.signals.tariffs.tou_tariff import TimeOfUseTariff
from utility_functions.signals.tariffs.tou_tariff import TimeOfUseTariff
from acnportal.acnsim.events import GaussianMixtureEvents
from acnportal.contrib.acnsim import StochasticNetwork
from acnportal.acnsim.network import ChargingNetwork
from modified_adacharge import modified_adacharge
from modified_simulator import modified_simulator
from acnportal.acnsim import analysis
# from modified_evse import *
from acnportal.acnsim.models.evse import get_evse_by_type

In [3]:
# This method won't take a evse_per_phase argument
def ev_fleet_level_2_network(transformer_cap=30):
    """ Configurable charging network for level-2 EVSEs connected line to line
        at 415 V.

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.
    """
    network = StochasticNetwork(early_departure=True)
    # network = ChargingNetwork()
    voltage = 415
    evse_type = 'AeroVironment'
    # evse_type = 'ClipperCreek'

    # Define the sets of EVSEs in the Caltech ACN.
    AB_ids = ['AB-{0}'.format(i) for i in range(3)]
    BC_ids = ['BC-{0}'.format(i) for i in range(3)]
    CA_ids = ['CA-{0}'.format(i) for i in range(3)]

    # print(AB_ids)

    # Add Caltech EVSEs
    for evse_id in AB_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, 30)
    for evse_id in BC_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, -90)
    for evse_id in CA_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, 150)

    # Add Caltech Constraint Set
    AB = acnsim.Current(AB_ids)
    BC = acnsim.Current(BC_ids)
    CA = acnsim.Current(CA_ids)

    # Define intermediate currents
    I3a = AB - CA
    I3b = BC - AB
    I3c = CA - BC
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 415
    print(primary_side_constr)
    secondary_side_constr = transformer_cap * 1000 / 3 / 230
    print(secondary_side_constr)
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network

#### Default Simulation Parameter

In [4]:
# How long each time discrete time interval in the simulation should be.
PERIOD = 5  # minutes

# Voltage of the network.
VOLTAGE = 415  # volts

# Default maximum charging rate for each EV battery.
DEFAULT_BATTERY_POWER = 11 # kW

#### Generate synthetic events for simulation

In [5]:
# Generate synthetic events based on JPL data provided by ACN-Sim.
class CustomUnpicklerJPLdata(pickle.Unpickler):
    def find_class(self, module, name):
        if name == "sklearn.mixture.gaussian_mixture":
            return sklearn.mixture.GaussianMixture
        if name == "GaussianMixture":
            return sklearn.mixture.GaussianMixture
        return super().find_class(module, name)
    
def get_synth_events(sessions_per_day):
    gmm = CustomUnpicklerJPLdata(open('./data/jpl_weekday_40.pkl', "rb")).load()


    # Generate a list of the number of sessions to draw for each day.
    # This generates 30 days of charging demands.
    # num_evs = [0]*2 + [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2 + \
    #           [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2

    # Generate sessions for 1 day (weekdays only)
    num_evs = [0]*2 + [sessions_per_day]*1

    # Note that because we are drawing from a distribution, some sessions will be
    # invalid, we ignore these sessions and remove the corresponding plugin events.
    gen = GaussianMixtureEvents(pretrained_model=gmm, duration_min=0.08334)

    synth_events = gen.generate_events(num_evs, PERIOD, VOLTAGE, DEFAULT_BATTERY_POWER)
    return synth_events

In [6]:
# 25 EV sessions per weekday
events_25_ev = get_synth_events(25)

c:\Users\s3955218\Anaconda3\envs\evsim\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.20.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
# Checking session details
session_copy = deepcopy(events_25_ev)
# print(type(session_copy))
session_list = list(session_copy.queue)
for session in session_list:
    print(session[0], session[1].ev._session_id, session[1].ev.arrival, \
          session[1].ev.departure, session[1].ev.requested_energy, session[1].event_type)

647 session_13 647 769 23.813524624175106 Plugin
655 session_16 655 761 11.736022912045913 Plugin
649 session_24 649 749 5.7460370312180284 Plugin
661 session_10 661 795 8.874595138386143 Plugin
656 session_20 656 781 14.061758304997117 Plugin
651 session_4 651 753 11.593674000748251 Plugin
653 session_5 653 752 12.266643922744063 Plugin
728 session_3 728 770 3.9508202691104692 Plugin
680 session_17 680 787 8.990253639702306 Plugin
665 session_1 665 786 57.547286680645485 Plugin
667 session_21 667 788 13.836313342163166 Plugin
664 session_11 664 776 7.669528501278178 Plugin
682 session_12 682 792 24.838053264193842 Plugin
669 session_6 669 715 16.651958125860936 Plugin
661 session_14 661 764 21.644529503779683 Plugin
771 session_15 771 797 8.491624289836865 Plugin
744 session_7 744 756 5.601233427359151 Plugin
758 session_8 758 786 13.607229967449486 Plugin
744 session_18 744 769 0.7299596660560134 Plugin
810 session_19 810 811 0.8203689087501793 Plugin
786 session_9 786 819 17.1670900

#### Algorithms and new equation

In [8]:
sch = {}
sch['Unctrl'] = algorithms.UncontrolledCharging()
sch['RR'] = algorithms.RoundRobin(algorithms.first_come_first_served, continuous_inc=1)

cost_min_obj = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost_with_pv),
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-6)
               ]

sch['MPC'] = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=200, max_recompute=1)

#### Run Simulation with New Equation

In [9]:
sims = dict()
def run_experiment(alg_name, cap):
    """ Run simulation for the events defined previously and the specified
        network / algorithm / events.
    """
    # Timezone of the ACN we are using.
    # timezone = pytz.timezone('America/Los_Angeles')
    # Australian timezone for the experiment
    timezone = pytz.timezone('Australia/Melbourne')

    start = timezone.localize(datetime(2024, 7, 31))
    end = timezone.localize(datetime(2024, 7, 31))

    cn = ev_fleet_level_2_network(transformer_cap=cap)

    alg = deepcopy(sch[alg_name])
    alg.max_rate_estimator = algorithms.SimpleRampdown()
    alg.estimate_max_rate = True
    experiment_events = deepcopy(events_25_ev)
    signals = {'tariff': TimeOfUseTariff('sce_tou_ev_4_march_2019')}

    sim = modified_simulator.Simulator(cn, alg, experiment_events, start, period=PERIOD, signals=signals, verbose=False)
    print("Running...")
    start_simulation = time.time()
    if alg_name == "MPC_Offline":
        alg.register_events(experiment_events)
        alg.solve()
    # if alg_name == "MPC":
    #     sim.high_priority_ev_sessions = ["session_2", "session_8", "session_19", "session_7"]
    # sim.high_priority_ev_sessions = ["session_5", "session_2", "session_19", "session_0", "session_6"]
    sim.run()
    print(f"Run time: {time.time() - start_simulation}")

    return sim

In [10]:
warnings.simplefilter("ignore")

capacities = list(range(60, 90, 5))
alg_names = ["Unctrl", "RR", "MPC"]

for cap in capacities:
    for alg_name in alg_names:
        config = (alg_name, cap)
        print(config)
        if config not in sims:
            sims[config] = run_experiment(*config)

('Unctrl', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 0.32482361793518066
('RR', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 1.8315024375915527
('MPC', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 11.127864122390747
('Unctrl', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 0.2831389904022217
('RR', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 1.8015499114990234
('MPC', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 16.40020775794983
('Unctrl', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 0.2845475673675537
('RR', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 1.9260859489440918
('MPC', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 11.120248556137085
('Unctrl', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 0.2837793827056885
('RR', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 1.8110589981079102
('MPC', 75)
60.2

In [ ]:
# result_dir = "results/sims/new_simulation"
# if not os.path.exists(result_dir):
#     os.makedirs(result_dir)
    
# for config, sim in sims.items():
#     name = "results/sims/new_simulation/{0}-{1}.json.gz".format(*config)
#     if not os.path.exists(name):
#         data = sim.to_json()
#         with gzip.GzipFile(name, 'w') as fout:
#             fout.write(json.dumps(data).encode('utf-8'))

#### Result Analysis

In [11]:
def calc_metrics(config, sim):
    metrics = {
        "Network Type": "three_phase",
        "Algorithm": config[0],
        "Capacity (kW)": config[1],
        "Energy Delivered (%)": analysis.proportion_of_energy_delivered(sim) * 100,
        "Demand met": analysis.proportion_of_demands_met(sim) * 100,
        "Max Utilization (%)": np.max(analysis.aggregate_power(sim)) / config[1] * 100,
        "energy_cost": analysis.energy_cost(sim),
        "total_energy_delivered": analysis.total_energy_delivered(sim),
        "Peak (kW)": np.max(analysis.aggregate_power(sim))
    }
    metrics["Current Unbalance"] = np.nanmean(analysis.current_unbalance(sim, ['Secondary {0}'.format(p) for p in 'ABC'], 'NEMA'))

    return metrics

In [12]:
metrics = pd.DataFrame(calc_metrics(config, sim) for config, sim in sims.items()).round(3)
metrics.rename(columns={"Capacity (kW)": "capacity"}, inplace=True)

In [13]:
metrics

,Network Type,Algorithm,capacity,Energy Delivered (%),Demand met,Max Utilization (%),energy_cost,total_energy_delivered,Peak (kW),Current Unbalance
0,three_phase,Unctrl,60,100.000,100.0,140.825,49.340,382.109,84.495,0.375
1,three_phase,RR,60,98.281,84.0,103.750,53.280,375.542,62.250,0.352
2,three_phase,MPC,60,97.837,76.0,94.767,56.443,373.844,56.860,0.327
3,three_phase,Unctrl,65,100.000,100.0,129.993,49.340,382.109,84.495,0.386
4,three_phase,RR,65,99.840,96.0,103.431,54.122,381.496,67.230,0.376
5,three_phase,MPC,65,91.720,48.0,98.646,48.738,350.471,64.120,0.414
6,three_phase,Unctrl,70,100.000,100.0,120.707,49.340,382.109,84.495,0.388
7,three_phase,RR,70,98.322,88.0,103.750,52.650,375.696,72.625,0.397
8,three_phase,MPC,70,97.780,68.0,94.864,55.351,373.625,66.405,0.335
9,three_phase,Unctrl,75,100.000,100.0,112.660,49.340,382.109,84.495,0.370


#### Simulation with different algorithms

In [14]:
sch = {}
sch['Unctrl'] = algorithms.UncontrolledCharging()
sch['LLF'] = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)

cost_min_obj = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost_with_pv),
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-6)
               ]

sch['MPC'] = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=200, max_recompute=1)

In [16]:
sims = dict()
# def run_experiment(alg_name, cap):
#     """ Run simulation for the events defined previously and the specified
#         network / algorithm / events.
#     """
#     # Timezone of the ACN we are using.
#     # timezone = pytz.timezone('America/Los_Angeles')
#     # Australian timezone for the experiment
#     timezone = pytz.timezone('Australia/Melbourne')

#     start = timezone.localize(datetime(2024, 7, 29))
#     end = timezone.localize(datetime(2024, 7, 29))

#     cn = ev_fleet_level_2_network(transformer_cap=cap)

#     alg = deepcopy(sch[alg_name])
#     alg.max_rate_estimator = algorithms.SimpleRampdown()
#     alg.estimate_max_rate = True
#     experiment_events = deepcopy(events_25_ev)
#     signals = {'tariff': TimeOfUseTariff('sce_tou_ev_4_march_2019')}

#     sim = modified_simulator.Simulator(cn, alg, experiment_events, start, period=PERIOD, signals=signals, verbose=False)
#     print("Running...")
#     start_simulation = time.time()
#     if alg_name == "MPC_Offline":
#         alg.register_events(experiment_events)
#         alg.solve()
#     # if alg_name == "MPC":
#     #     sim.high_priority_ev_sessions = ["session_2", "session_8", "session_19", "session_7"]
#     # sim.high_priority_ev_sessions = ["session_5", "session_2", "session_19", "session_0", "session_6"]
#     sim.run()
#     print(f"Run time: {time.time() - start_simulation}")

#     return sim

In [17]:
warnings.simplefilter("ignore")

capacities = list(range(60, 90, 5))
alg_names = ["Unctrl", "LLF", "MPC"]

for cap in capacities:
    for alg_name in alg_names:
        config = (alg_name, cap)
        print(config)
        if config not in sims:
            sims[config] = run_experiment(*config)

('Unctrl', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 0.331146240234375
('LLF', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 0.9871039390563965
('MPC', 60)
48.19277108433735
86.95652173913044
Running...
Run time: 12.07369613647461
('Unctrl', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 0.28993797302246094
('LLF', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 0.8795766830444336
('MPC', 65)
52.208835341365464
94.20289855072464
Running...
Run time: 11.818645238876343
('Unctrl', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 0.334075927734375
('LLF', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 0.9724912643432617
('MPC', 70)
56.22489959839357
101.44927536231883
Running...
Run time: 14.24342942237854
('Unctrl', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 0.3026156425476074
('LLF', 75)
60.24096385542169
108.69565217391305
Running...
Run time: 0.7632389068603516
('MPC', 75)
60.

In [18]:
metrics = pd.DataFrame(calc_metrics(config, sim) for config, sim in sims.items()).round(3)
metrics.rename(columns={"Capacity (kW)": "capacity"}, inplace=True)

In [19]:
metrics

,Network Type,Algorithm,capacity,Energy Delivered (%),Demand met,Max Utilization (%),energy_cost,total_energy_delivered,Peak (kW),Current Unbalance
0,three_phase,Unctrl,60,100.000,100.0,140.825,49.340,382.109,84.495,0.406
1,three_phase,LLF,60,99.733,84.0,89.583,54.547,381.087,53.750,0.267
2,three_phase,MPC,60,90.566,52.0,99.258,52.932,346.063,59.555,0.401
3,three_phase,Unctrl,65,100.000,100.0,129.993,49.340,382.109,84.495,0.382
4,three_phase,LLF,65,99.772,84.0,89.400,56.440,381.239,58.110,0.382
5,three_phase,MPC,65,95.376,72.0,95.138,53.929,364.439,61.840,0.321
6,three_phase,Unctrl,70,100.000,100.0,120.707,49.340,382.109,84.495,0.408
7,three_phase,LLF,70,99.812,88.0,88.350,49.729,381.391,61.845,0.337
8,three_phase,MPC,70,97.883,84.0,96.643,55.626,374.019,67.650,0.298
9,three_phase,Unctrl,75,100.000,100.0,112.660,49.340,382.109,84.495,0.379
